In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import tensorflow as tf
path = r'..\\embeddings\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 500
with open(path + "m_base2021_500d_150e_w", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

260377


In [3]:
def find_candidate(sim,id_a,id_a_star,id_b):
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    
    if(myList[0] != 0):
        index1 = np.argmax(sim[:myList[0]])
    else:
        index1 = None
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim2)
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim3)
        index3 +=  myList[1]+1
    else:
        index3 = None
    if(myList[2]+1 < len(sim)):
        index4 = np.argmax(sim[myList[2]+1:])
        index4 +=  myList[2]+1
    else:
        index4 = None
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    return index

In [4]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)

In [5]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list


print(find_nearest_k('batman',6))#this should produce one, for the same vectors
print(find_nearest_k('hogwarts',6))
print(find_nearest_k('turing',6))
print(find_nearest_k('florida',6))
#print(find_nearest_k('object-oriented',6))
print(find_nearest_k('dancing',6))

[('batman', 0.9999999999999998), ('superman', 0.6273830433443), ('joker', 0.5280659207871355), ('catwoman', 0.5272311039529596), ('superhero', 0.48298571425138853), ('nightwing', 0.4820262185753492)]
[('hogwarts', 0.9999999999999996), ('dumbledore', 0.43441635133944034), ('rajdhani', 0.38506617599396964), ('humsafar', 0.3845480987641291), ('tsukuba', 0.3831666272952117), ('kanchenjunga', 0.38303093520430687)]
[('turing', 0.9999999999999999), ('deterministic', 0.39966130556913976), ('alda', 0.38306674441227645), ('machine', 0.3731029957065046), ('vending', 0.369334867167193), ('dershowitz', 0.36125806253429643)]
[('florida', 1.0000000000000007), ('miami', 0.5591805687086543), ('alabama', 0.5536634197803354), ('carolina', 0.5380821123573977), ('gainesville', 0.5352767614086814), ('jacksonville', 0.5345697933943235)]
[('dancing', 0.9999999999999988), ('singing', 0.634503461930655), ('dance', 0.6099806234145523), ('dancers', 0.5979258609182083), ('dancer', 0.5379875236804683), ('dances', 0

## 3cosAdd 

In [6]:

def three_cos_predict_np(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix[id_a,:]
    weight_b = matrix[id_b,:]
    weight_a_star = matrix[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# 3cosAdd normalised befor arithmetic

In [7]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

## 3cos Mult

In [8]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        print(a in id_dict)
        print(a_star in id_dict)
        print(b in id_dict)
        print(b_star in id_dict)
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star) * sim_01(matrix_normalized,weight_b) 
    denominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# Test all

In [9]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0,0]
count_syn_sucess      = [0,0,0]

errors = []


files = os.listdir('.\\datasets\\question-data')
for idx,file_name in enumerate(files):
    print(file_name)
    with open('.\\datasets\\question-data\\'+file_name,'r') as file:
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
        
        for line_id,line in enumerate(lines):
            if (line_id % 100 == 0):
                print(line_id)
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            #predicted_3cos_add      = three_cos_predict_np(a,a_star,b,b_star)
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            #print(a,a_star,b,b_star)
            #print(predicted_3cos_add,predicted_3cos_add_norm,predicted_3cos_mult)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_sem_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_sem_sucess[2] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_syn_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_syn_sucess[2] += 1
        print(local_sucesses[0]/float(local_tasks_count))
        print(local_sucesses[1]/float(local_tasks_count))
        print(local_sucesses[2]/float(local_tasks_count))
        print('-------------------')

print('semantical')

print(count_sem_sucess[0]/float(count_sem_questions))
print(count_sem_sucess[1]/float(count_sem_questions))
print(count_sem_sucess[2]/float(count_sem_questions))
        
print('syntactical')
print(count_syn_sucess[0]/float(count_sem_questions))
print(count_syn_sucess[1]/float(count_sem_questions))
print(count_syn_sucess[2]/float(count_sem_questions))

print('overall')
print((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[2]+count_sem_sucess[2])/float(count_sem_questions+count_sem_questions))

capital-common-countries.txt
0
100
200
300
400
500
0.0
0.9782608695652174
0.974308300395257
-------------------
capital-world.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
0.0
0.8890362511052167
0.898762157382847
-------------------
city-in-state.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
0.0
0.6903121199837859
0.7044993919740575
-------------------
currency.txt
0
True
True
True
False
True
True
True
False
True
True
True
False
100
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
200
True
True
True
False
True
True
True
False
True
True
True
False
True
True
True
False
300
True
True
True
False
True
True
True
False
True
True
True
False
400
True
True
True
False
True
True
True
False
True
True
True
False
True
True


In [10]:
print('semantical')

print(count_sem_sucess[0]/float(count_sem_questions))
print(count_sem_sucess[1]/float(count_sem_questions))
print(count_sem_sucess[2]/float(count_sem_questions))
        
print('syntactical')
print(count_syn_sucess[0]/float(count_sem_questions))
print(count_syn_sucess[1]/float(count_sem_questions))
print(count_syn_sucess[2]/float(count_sem_questions))

print('overall')
print((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[2]+count_sem_sucess[2])/float(count_sem_questions+count_sem_questions))

semantical
0.0
0.7529597474348856
0.7654752508738302
syntactical
0.0
0.6566692975532754
0.6850828729281768
overall
0.0
0.7048145224940805
0.7252790619010034


In [11]:
import winsound
winsound.Beep(440, 500)

Spearman Rank

In [13]:
with open(r'.\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            #print(id1)
            #print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = sim_01(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))

linetiger	cat	7.35

tiger cat
tf.Tensor(0.6934915120768786, shape=(), dtype=float64) 0.735
linetiger	tiger	10.00

tiger tiger
tf.Tensor(1.0, shape=(), dtype=float64) 1.0
lineplane	car	5.77

plane car
tf.Tensor(0.6795920009156688, shape=(), dtype=float64) 0.577
linetrain	car	6.31

train car
tf.Tensor(0.7417733554379867, shape=(), dtype=float64) 0.631
linetelevision	radio	6.77

television radio
tf.Tensor(0.7969138494723481, shape=(), dtype=float64) 0.6769999999999999
linemedia	radio	7.42

media radio
tf.Tensor(0.7113135868329117, shape=(), dtype=float64) 0.742
linebread	butter	6.19

bread butter
tf.Tensor(0.7660938041078724, shape=(), dtype=float64) 0.619
linecucumber	potato	5.92

cucumber potato
tf.Tensor(0.6866004482867368, shape=(), dtype=float64) 0.592
linedoctor	nurse	7.00

doctor nurse
tf.Tensor(0.7046488948180128, shape=(), dtype=float64) 0.7
lineprofessor	doctor	6.62

professor doctor
tf.Tensor(0.6983823929583671, shape=(), dtype=float64) 0.662
linestudent	professor	6.81

student

tf.Tensor(0.6430172461346806, shape=(), dtype=float64) 0.475
lineministry	culture	4.69

ministry culture
tf.Tensor(0.6880021199605177, shape=(), dtype=float64) 0.46900000000000003
linesmart	student	4.62

smart student
tf.Tensor(0.6236386580452966, shape=(), dtype=float64) 0.462
lineinvestigation	effort	4.59

investigation effort
tf.Tensor(0.6628234843775563, shape=(), dtype=float64) 0.45899999999999996
lineimage	surface	4.56

image surface
tf.Tensor(0.6483437273914168, shape=(), dtype=float64) 0.45599999999999996
linelife	term	4.50

life term
tf.Tensor(0.6824276353400162, shape=(), dtype=float64) 0.45
linestart	match	4.47

start match
tf.Tensor(0.6719094189663326, shape=(), dtype=float64) 0.44699999999999995
linecomputer	news	4.47

computer news
tf.Tensor(0.6096913040815404, shape=(), dtype=float64) 0.44699999999999995
lineboard	recommendation	4.47

board recommendation
tf.Tensor(0.6375364989277942, shape=(), dtype=float64) 0.44699999999999995
linelad	brother	4.46

lad brother
tf.Tenso

In [14]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.6282645902228762, pvalue=1.0846964368661613e-23)